Q1. Getting the embeddings model

In [16]:
from sentence_transformers import SentenceTransformer

In [17]:
embedding_model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(embedding_model_name)

In [18]:
user_question = "I just discovered the course. Can I still join it?"
user_question_embedding = embedding_model.encode(user_question)

In [19]:
user_question_embedding[0]

0.078222655

In [20]:
import requests

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [21]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [22]:
course_name = "machine-learning-zoomcamp"
filtered_documents = [item for item in documents if item["course"] == course_name]

In [23]:
len(filtered_documents)

375

Q2. Creating the embeddings

In [24]:
from tqdm.auto import tqdm

embeddings = []

for doc in tqdm(filtered_documents):
    qa_text = f"{doc['question']} {doc['text']}"
    doc_embeddings = embedding_model.encode(qa_text)
    embeddings.append(doc_embeddings)

embeddings[0]

  0%|          | 0/375 [00:00<?, ?it/s]

100%|██████████| 375/375 [00:21<00:00, 17.42it/s]


array([ 8.80591124e-02,  1.55936033e-02,  7.92557523e-02,  2.52757687e-02,
        7.55764917e-02, -3.90596800e-02, -4.13814038e-02,  2.52916943e-02,
        2.43241936e-02,  3.62587790e-03, -7.28286849e-03, -3.28751504e-02,
        6.12956285e-02, -5.71100414e-02,  1.16774449e-02, -1.79441702e-02,
        4.49206121e-02, -5.41606732e-02, -1.92255690e-03,  1.48329614e-02,
        7.91351125e-03, -3.43127102e-02,  1.21481167e-02,  1.27186347e-02,
       -1.60092004e-02,  7.12132733e-03,  1.58389080e-02, -3.27576447e-04,
        3.01294727e-03, -4.58917543e-02, -1.34771131e-03,  2.01150123e-02,
        2.33724173e-02, -4.28203307e-03, -3.09383031e-02,  1.94304925e-03,
       -1.88219491e-02,  5.35158208e-03,  1.29352380e-02, -3.23958807e-02,
        3.10556982e-02,  5.12981601e-03, -3.24515800e-04, -3.15542109e-02,
        1.93377640e-02, -6.38601482e-02, -3.05381212e-02, -6.27885312e-02,
       -1.74859818e-02,  3.52948979e-02, -2.98071187e-02,  7.45345950e-02,
       -2.06182245e-02, -

In [25]:
import numpy as np

In [26]:
X = np.array(embeddings)

# ↓ (rows) → (columns)
X.shape

(375, 768)

Q3. Search

In [27]:
# Select a vector (for example, the first vector in X)
vector = X[0]

# The vectors returned from the embedding model are already normalized 
# (you can check it by computing a dot product of a vector with itself - it should return 1.0)
dot_product = np.dot(vector, vector)

dot_product

1.0

In [34]:
scores = X.dot(user_question_embedding)
print(scores)
print()
print(f"The answer to Q3.1: {scores.max()}")

[ 2.89217323e-01  4.35050964e-01  2.20572680e-01  1.28085867e-01
  3.28754723e-01  4.58999753e-01  4.48930800e-01  2.70170838e-01
  2.13974997e-01  3.39745611e-01  1.63339078e-01  3.15917820e-01
  1.53621100e-02  4.14240539e-01  6.50657535e-01  4.12786603e-01
  2.02390447e-01  4.71707731e-01  3.74821365e-01  3.53319347e-01
  4.50558484e-01  1.85430810e-01  3.45527023e-01  8.22948143e-02
 -2.19077673e-02 -4.64250520e-02  8.60690847e-02  9.87197980e-02
 -2.14188341e-02  1.28751695e-02  2.54869889e-02  3.66865806e-02
  4.56339307e-02 -2.05279551e-02  1.82662327e-02  3.39788646e-02
  1.66507531e-03  1.03210807e-02  3.40170003e-02 -2.65230574e-02
 -4.11910005e-02 -4.97983135e-02 -6.20346777e-02 -7.63562508e-03
  4.61617671e-02  6.95954915e-03  4.77613285e-02 -9.97651368e-04
 -5.60125411e-02 -5.96207008e-02  2.05209315e-01 -1.16968520e-01
 -2.34159678e-02 -1.52457990e-02 -6.83521200e-03 -8.61714333e-02
  2.02636980e-02 -2.51591727e-02  4.21458855e-04 -3.79212871e-02
 -3.21440250e-02  2.48612